## Alcholic Anonymous News articles extraction from BBC ( USA & UK ). Data Extraction of following parameters
- Headline
- Description
- Author
- Published Date
- Category
- Publication
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions.

### Lists for Alcholic Anonymous News Articles extracted data

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, category, publication = [], [], [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results

In [3]:
keyword = 'Alcoholics Anonymous site:www.bbc.co.uk'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### Finding the total no.of.pages by total no.of articles from google search results

In [5]:
keyword = 'Alcoholics Anonymous site:www.bbc.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [6]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [7]:
urls = list(dict.fromkeys(urls))
print("Total Extracted URL's are" + ' : ' + str(len(urls)), type(urls))

Total Extracted URL's are : 491 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [8]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('.date')['data-datetime'])
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(' '.join(soup.select_one('.storyText').text.split()).replace('\n', '').strip())
            except:
                news.append(' '.join(article.text.split()).replace("\'\'"," ").replace("\'", "").replace(" / ", ""))
        except:
            news.append(None)
            
        # Extracts the news category
        try:
            category.append(article.meta_data['category'])
        except:
            category.append(None)
            
        # Extracts the news publication
        try:
            publication.append(article.meta_data['og']['site_name'])
        except:
            publication.append(None)

        # Extracts Keywords and Summaries
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
                        
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        category.append(None)
        publication.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\GM\AppData\Local\Temp\jieba.cache
Loading model cost 1.6518609523773193 seconds.
Prefix dict has been built succesfully.


Wall time: 50min 12sc.com/news/localnews/2639997-portslade-by-sea/0ng=english-caout-drunkure-a-workaholicances-filmed-on-mobileoblems


### Checking Array Length of each list to create DataFrame

In [9]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(category), len(publication), len(news), len(keywords), len(summaries), len(urls))

491 491 491 491 491 491 491 491 491 491


### Creating a csv file after checking array length and droping the missing values from the dataset

In [10]:
if len(headlines) == len(descriptions) == len(authors) == len(dates) == len(news) == len(publication) == len(keywords) == len(summaries) == len(urls) == len(category):
    tbl = pd.DataFrame({'Headlines' : headlines,
                        'Descriptions' : descriptions,
                        'Authors' : authors,
                        'Published_Dates' : dates,
                        'Publication' : publication,
                        'Articles' : news,
                        'category' : category,
                        'Keywords' : keywords,
                        'Summaries' : summaries,
                        'Source_URLs' : urls})
    tbl.dropna()
    path = 'C:\\Users\\GM\\Desktop\\!Code!\\CDRI\\Alcoholics Anonymous\\Data Extraction\\#Dataset\\'
    tbl.to_csv(path+'BBC.csv', index=False)
else:
    print('Array lenght does not match!')

tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Publication,Articles,category,Keywords,Summaries,Source_URLs
0,Object : Alcoholics Anonymous Badge,A History of the World is a partnership betwee...,None,None,None,The contributors father was a member of Alcoho...,{},"[unless, content, views, anonymous, badge, alc...",The contributor's father was a member of Alcoh...,http://www.bbc.co.uk/ahistoryoftheworld/object...
1,Every university 'needs' Alcoholics Anonymous ...,Alcoholics Anonymous says the number of younge...,None,2018-07-31 16:54:32+00:00,BBC News,Image copyright Getty Images Alcoholics Anonym...,{},"[student, really, anonymous, younger, wales, m...",Image copyright Getty ImagesAlcoholics Anonymo...,https://www.bbc.co.uk/news/amp/uk-wales-45023075
2,What happens in an Alcoholics Anonymous meeting?,Members of Alcoholics Anonymous are marking it...,None,10 June 2010,BBC News,Image caption The true nature of alcoholism is...,{},"[happens, anonymous, bad, group, meeting, alco...",Image caption The true nature of alcoholism is...,https://www.bbc.co.uk/news/10280806
3,The many groups that have copied Alcoholics An...,Alcoholics Anonymous was founded 80 years ago....,None,9 June 2015,BBC News,Image copyright Other Image caption An AA meet...,{},"[copied, anonymous, steps, 12step, meeting, al...","But Peter hasn't tasted alcohol in 34 years, a...",https://www.bbc.co.uk/news/magazine-33049093
4,Alcoholics Anonymous,,None,None,BBC,,{},"[alcoholics, anonymous]",,https://www.bbc.co.uk/programmes/topics/Alcoho...


In [11]:
tbl.shape

(491, 10)